In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps trl peft accelerate bitsandbytes -q
!pip install --upgrade --force-reinstall pyarrow
!pip install datasets==4.3.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (47.7 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 22.0.0
    Uninstalling pyarrow-22.0.0:
      Successfully uninstalled pyarrow-22.0.0


In [2]:
%cd /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/datasets

/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/datasets


In [3]:
!git clone https://huggingface.co/datasets/Naholav/CodeGen-Deep-5K

fatal: destination path 'CodeGen-Deep-5K' already exists and is not an empty directory.


In [4]:
!git clone https://huggingface.co/datasets/Naholav/CodeGen-Diverse-5K

fatal: destination path 'CodeGen-Diverse-5K' already exists and is not an empty directory.


In [5]:
%cd /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/model

/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/model


In [6]:
!git clone https://huggingface.co/Qwen/Qwen2.5-Coder-1.5B-Instruct

fatal: destination path 'Qwen2.5-Coder-1.5B-Instruct' already exists and is not an empty directory.


In [7]:
%cd /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA

/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA


In [8]:
import os
import json
import torch
from datetime import datetime
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from transformers.trainer_callback import TrainerCallback

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [9]:
TRAINING_MODE = "deep"

USE_REASONING = False

DEEP_DATASET = "Naholav/CodeGen-Deep-5K"
DIVERSE_DATASET = "Naholav/CodeGen-Diverse-5K"

In [10]:
if USE_REASONING:
  SYSTEM_PROMPT = "You are an expert Python programmer. Please read the problem carefully before writing any Python code."
else:
  SYSTEM_PROMPT = "You are an expert programmer. Use <think> tags for reasoning before writing code."

In [11]:
LORA_R = 32
LORA_ALPHA = 64  # r * 2
LORA_DROPOUT = 0.1

In [12]:
LEARNING_RATE = 2e-4
EPOCHS = 3
BATCH_SIZE = 16
GRADIENT_ACCUMULATION = 1
MAX_LENGTH = 1024 if not USE_REASONING else 8192

WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.03
SEED = 42

In [13]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
DIRECTORY_PATH = "/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA"
OUTPUT_DIR = f"{DIRECTORY_PATH}/training_models/{TRAINING_MODE}/{TRAINING_MODE}_{timestamp}"
LOG_DIR = f"{DIRECTORY_PATH}/logs/{TRAINING_MODE}/{TRAINING_MODE}_{timestamp}"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [14]:
class LogCallback(TrainerCallback):
    """Loss'ları hem ekrana hem dosyaya yaz"""
    def __init__(self, log_file):
        self.log_file = log_file

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            with open(self.log_file, 'a') as f:
                log_entry = {
                    "step": state.global_step,
                    "epoch": round(state.epoch, 2) if state.epoch else 0,
                    **logs
                }
                f.write(json.dumps(log_entry) + "\n")

            if "loss" in logs:
                print(f"📊 [Step {state.global_step}] Train Loss: {logs['loss']:.4f}")
            if "eval_loss" in logs:
                print(f"📈 [Step {state.global_step}] Eval Loss: {logs['eval_loss']:.4f}")

In [15]:
print("Model yükleniyor...")

max_seq_length = MAX_LENGTH
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-Coder-1.5B-Instruct",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=False,
)

print("Model yüklendi!")
print(f"Max sequence length: {max_seq_length}")
print(f"Device: {model.device}")

Model yükleniyor...
==((====))==  Unsloth 2025.12.1: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

Model yüklendi!
Max sequence length: 1024
Device: cuda:0


In [16]:
FastLanguageModel.for_inference(model)
if not USE_REASONING:
  messages = [
      {"role": "system", "content": "You are an expert Python programmer. Please read the problem carefully before writing any Python code."},
      {"role": "user", "content": "Write a Python function to check if a number is prime."}
  ]
else:
  messages = [
      {"role": "system", "content": "You are an expert programmer. Use <think> tags for reasoning before writing code."},
      {"role": "user", "content": "Write a Python function to check if a number is prime."}
  ]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1024)
print("BAŞLANGIÇ MODELİ CEVABI:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


BAŞLANGIÇ MODELİ CEVABI:
system
You are an expert Python programmer. Please read the problem carefully before writing any Python code.
user
Write a Python function to check if a number is prime.
assistant
Certainly! Below is a Python function that checks if a given number is prime:

```python
def is_prime(n):
    # Check for edge cases: 0, 1, and negative numbers are not prime
    if n <= 1:
        return False
    
    # Check for divisibility from 2 up to the square root of n
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    
    # If no divisors were found, the number is prime
    return True

# Example usage:
print(is_prime(7))  # Output: True
print(is_prime(10)) # Output: False
```

### Explanation:
1. **Edge Cases**: The function first checks if the input `n` is less than or equal to 1. If so, it returns `False` because 0 and 1 are not prime numbers.
2. **Divisibility Check**: It then iterates from 2 up to the square root of `n`. This is

In [17]:
print("\nLoRA uygulanıyor...")
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
)

print("LoRA uygulandı!")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.



LoRA uygulanıyor...


Unsloth 2025.12.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


LoRA uygulandı!


In [18]:
print("Dataset yükleniyor...")
dataset_name = DEEP_DATASET if TRAINING_MODE == "deep" else DIVERSE_DATASET
print(f"Dataset: {dataset_name}")

dataset = load_dataset(dataset_name)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
print(f"Train: {len(dataset['train'])} örnekler")
print(f"Test: {len(dataset['test'])} örnekler")

Dataset yükleniyor...
Dataset: Naholav/CodeGen-Deep-5K


README.md: 0.00B [00:00, ?B/s]

CodeGen-Deep-5K.jsonl:   0%|          | 0.00/55.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Train: 4500 örnekler
Test: 500 örnekler


In [19]:
def formatting_func(examples):
    inputs = examples["input"]
    outputs = examples["output"] if USE_REASONING else examples["solution"]

    texts = []

    for input_text, output_text in zip(inputs, outputs):
        text = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        text += f"<|im_start|>user\n{input_text}<|im_end|>\n"
        text += f"<|im_start|>assistant\n{output_text}<|im_end|>"
        texts.append(text)

    return texts

In [20]:
print("Training ayarları hazırlanıyor...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,

    # Optimizer & Scheduler
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    optim="adamw_8bit",
    lr_scheduler_type="cosine",

    # Logging & Saving
    logging_steps=20,
    eval_steps=100,
    save_steps=100,
    save_total_limit=3,

    # Evaluation & Model Selection
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Optimization
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    max_grad_norm=1.0,

    # Misc
    report_to="tensorboard",
    seed=SEED,
    dataloader_num_workers=2,
)

print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"Precision: {'BF16' if is_bfloat16_supported() else 'FP16'}")

Training ayarları hazırlanıyor...
Effective batch size: 16
Precision: BF16


In [21]:
print("\nTrainer hazırlanıyor...")
log_file = os.path.join(LOG_DIR, "training_log.jsonl")
log_callback = LogCallback(log_file)
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    max_seq_length=max_seq_length,
    args=training_args,
    callbacks=[log_callback, early_stopping],
    packing=False,
    formatting_func=formatting_func
)

config = {
    "training_mode": TRAINING_MODE,
    "dataset": dataset_name,
    "use_reasoning": USE_REASONING,
    "base_model": "Qwen2.5-Coder-1.5B-Instruct",
    "framework": "unsloth",

    "lora_config": {
        "r": LORA_R,
        "alpha": LORA_ALPHA,
        "dropout": LORA_DROPOUT,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"]
    },

    "training_config": {
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "gradient_accumulation": GRADIENT_ACCUMULATION,
        "effective_batch_size": BATCH_SIZE * GRADIENT_ACCUMULATION,
        "max_length": MAX_LENGTH,
        "weight_decay": WEIGHT_DECAY,
        "warmup_ratio": WARMUP_RATIO,
        "optimizer": "adamw_8bit",
        "scheduler": "cosine",
        "early_stopping_patience": 2,
        "seed": SEED
    }
}

with open(os.path.join(OUTPUT_DIR, "config.json"), 'w') as f:
    json.dump(config, f, indent=2)

print("Trainer hazır!")


Trainer hazırlanıyor...


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/4500 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/500 [00:00<?, ? examples/s]

Trainer hazır!


In [22]:
print("EĞİTİM BAŞLIYOR!")
gpu_stats = torch.cuda.get_device_properties(0)
print(f"GPU: {gpu_stats.name}")
print(f"Total Memory: {round(gpu_stats.total_memory / 1024 / 1024 / 1024, 2)} GB")
print(f"Compute Capability: {gpu_stats.major}.{gpu_stats.minor}\n")

trainer_stats = trainer.train()

print("EĞİTİM TAMAMLANDI!")


The model is already on multiple devices. Skipping the move to device specified in `args`.


EĞİTİM BAŞLIYOR!
GPU: NVIDIA A100-SXM4-80GB
Total Memory: 79.32 GB
Compute Capability: 8.0



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 3 | Total steps = 846
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 36,929,536 of 1,580,643,840 (2.34% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.851300,0.799587
200,0.676900,0.648409
300,0.394600,0.472222
400,0.274100,0.324532
500,0.190900,0.204627
600,0.105600,0.160490
700,0.092500,0.142763
800,0.093400,0.136806


📊 [Step 20] Train Loss: 1.1338
📊 [Step 40] Train Loss: 0.8932
📊 [Step 60] Train Loss: 0.8667
📊 [Step 80] Train Loss: 0.8249
📊 [Step 100] Train Loss: 0.8513
📈 [Step 100] Eval Loss: 0.7996
📊 [Step 120] Train Loss: 0.8075
📊 [Step 140] Train Loss: 0.7608
📊 [Step 160] Train Loss: 0.7115
📊 [Step 180] Train Loss: 0.6666
📊 [Step 200] Train Loss: 0.6769
📈 [Step 200] Eval Loss: 0.6484
📊 [Step 220] Train Loss: 0.6575
📊 [Step 240] Train Loss: 0.5893
📊 [Step 260] Train Loss: 0.5744
📊 [Step 280] Train Loss: 0.5122
📊 [Step 300] Train Loss: 0.3946
📈 [Step 300] Eval Loss: 0.4722
📊 [Step 320] Train Loss: 0.3774
📊 [Step 340] Train Loss: 0.3579
📊 [Step 360] Train Loss: 0.3319
📊 [Step 380] Train Loss: 0.3078
📊 [Step 400] Train Loss: 0.2741
📈 [Step 400] Eval Loss: 0.3245
📊 [Step 420] Train Loss: 0.2636
📊 [Step 440] Train Loss: 0.2284
📊 [Step 460] Train Loss: 0.2035
📊 [Step 480] Train Loss: 0.2255
📊 [Step 500] Train Loss: 0.1909
📈 [Step 500] Eval Loss: 0.2046
📊 [Step 520] Train Loss: 0.1698
📊 [Step 540] Trai

In [23]:
print("\nModel kaydediliyor...")
lora_dir = os.path.join(DIRECTORY_PATH, "lora_adapter")
model.save_pretrained(lora_dir)
tokenizer.save_pretrained(lora_dir)
print(f"LoRA adapter: {lora_dir}")


Model kaydediliyor...
LoRA adapter: /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/deep/deep_20251208_134507/lora_adapter


In [24]:
print("\nMerged model oluşturuluyor...")
merged_dir = os.path.join(OUTPUT_DIR, "merged_model")
model.save_pretrained_merged(merged_dir, tokenizer, save_method="merged_16bit")
print(f"Merged model (16bit): {merged_dir}")


Merged model oluşturuluyor...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/deep/deep_20251208_134507/merged_model`: 100%|██████████| 1/1 [00:07<00:00,  7.98s/it]


Successfully copied all 1 files from cache to `/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/deep/deep_20251208_134507/merged_model`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:16<00:00, 16.44s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/deep/deep_20251208_134507/merged_model`
Merged model (16bit): /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/deep/deep_20251208_134507/merged_model


In [25]:
print("\nHızlı inference testi yapılıyor...\n")

FastLanguageModel.for_inference(model)  # Inference mode

test_prompt = """Write a Python function that takes a list of integers and returns the sum of all even numbers."""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": test_prompt}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=1024,
    temperature=0.6,
    do_sample=True,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("TEST ÇIKTISI:")
print(response)

print("\nGPU Memory Kullanımı:")
print(f"Allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

print("\n\nTüm işlemler tamamlandı! Başarılar!")


Hızlı inference testi yapılıyor...

TEST ÇIKTISI:
system
You are an expert programmer. Use <think> tags for reasoning before writing code.
user
Write a Python function that takes a list of integers and returns the sum of all even numbers.
assistant
def main():
    # Read input from stdin
    data = input().split()
    
    # Convert string list to integer list
    numbers = list(map(int, data))
    
    # Initialize sum variable
    total_sum = 0
    
    # Iterate through each number in the list
    for num in numbers:
        # Check if the number is even
        if num % 2 == 0:
            # Add it to the total sum
            total_sum += num
    
    # Print the final result
    print(total_sum)

if __name__ == "__main__":
    main()

GPU Memory Kullanımı:
Allocated: 3.25 GB
Reserved: 3.37 GB


Tüm işlemler tamamlandı! Başarılar!


In [26]:
import random

num_samples = 3
test_indices = random.sample(range(len(dataset["test"])), num_samples)
test_samples = dataset["test"].select(test_indices)

print("Test Analizi")

FastLanguageModel.for_inference(model)

for i, sample in enumerate(test_samples):
  input_text = sample["input"]
  ground_truth = sample["solution"]

  messages = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": input_text}
  ]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to("cuda")

  outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.9,
        do_sample=True
  )

  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(f"\nÖRNEK {i+1}:")
  print(f"SORU:\n{input_text[:200]}...")
  print(f"|nMODEL TAHMİNİ:\n{prediction.split('assistant')[-1].strip()}")
  print(f"GERÇEK CEVAP (Referans):\n{ground_truth[:200]}...")

Test Analizi

ÖRNEK 1:
SORU:
You have been given an array A of size N and an integer K. This array consists of N integers ranging from 1 to 10^7. Each element in this array is said to have a Special Weight. The special weight of ...
|nMODEL TAHMİNİ:
import sys

def main():
    data = sys.stdin.read().split()
    n = int(data[0])
    k = int(data[1])
    arr = list(map(int, data[2:2+n]))
    
    # Create list of (weight, original_value) pairs
    pairs = []
    for num in arr:
        weight = num % k
        if weight == 0:
            pairs.append((k, num))
        else:
            pairs.append((weight, num))
    
    # Sort by weight descending, then by value ascending
    pairs.sort(key=lambda x: (-x[0], x[1]))
    
    # Extract values from sorted pairs
    result = [str(pair[1]) for pair in pairs]
    print(" ".join(result))

if __name__ == "__main__":
    main()
GERÇEK CEVAP (Referans):
import sys

def main():
    data = sys.stdin.read().split()
    n = int(data[0])
    k = int